##### 도서 리뷰 크롤링
* 교보문고, 베스트셀러 도서 기준으로 도서 리뷰 가져오기

In [ ]:
# 감정 분석하기 위해 설치
! pip install transformers

In [ ]:
! pip install torch

In [ ]:
# 감정 분석하기 위해 설치
! pip show tensorflow

In [ ]:
# 확인
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.6.0+cpu
False


In [6]:
import os

os.environ["TRANSFORMERS_NO_TF"] = "1"

In [ ]:
# 감정 분석 실험용
from transformers import pipeline


# Huggingface에서 감정 분석 파이프라인을 불러옴
# PyTorch로 강제 지정
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="snunlp/KR-FinBert-SC",
    framework="pt"
)

# 분석할 한국어 텍스트
text = "이 영화 정말 재미있어요!"

# 감정 분석 예측
result = sentiment_analyzer(text)
print(result)


In [13]:
import requests
from bs4 import BeautifulSoup
import time
from transformers import pipeline


# Huggingface에서 감정 분석 파이프라인을 불러옴
# PyTorch로 강제 지정
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="snunlp/KR-FinBert-SC",
    framework="pt",
    device=-1
)


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36'
}

# 1. 베스트셀러 페이지에서 도서 목록 추출
def get_bestseller_books():
    url = 'https://store.kyobobook.co.kr/bestseller/realtime'
    res = requests.get(url, headers=headers)
    
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        a_tag_list = soup.select('a.prod_link.line-clamp-2.font-medium.text-black.hover\\:underline.fz-16.mt-2')
        best_books = []

        for a_tag in a_tag_list:
            title = a_tag.text.strip()
            link = a_tag['href']
            best_books.append((title, link))
        
        return best_books


# 2. 각 도서 상세 페이지에서 리뷰 추출
def get_book_reviews(book_url, max_pages=3):
    reviews = []

    # url에서 product_id 추출
    product_id = book_url.split('/')[-1]

    for page in range(1, max_pages + 1):
        # review json url
        review_url = f'https://product.kyobobook.co.kr/api/review/list?page={page}&pageLimit=10&reviewSort=001&revwPatrCode=002&saleCmdtid={product_id}'
        
        res = requests.get(review_url, headers=headers)
        
        if not res.ok:
            break

        data = res.json()
        items = data.get("data", {}).get("reviewList", [])
        if not items:
            break

        for item in items:
            reviews.append(item['revwCntt'].strip())

        # 방지!!
        time.sleep(0.3)
    
    return reviews

# 실행
max_length = 100
bestsellers = get_bestseller_books()
print("베스트셀러 5권만 출력 및 리뷰 수집")

for title, link in bestsellers[:10]:  # 상위 10개만
    print(f"\n📘 {title}")
    reviews = get_book_reviews(link)
    for i, review in enumerate(reviews[:3], 1):  # 일단 리뷰 3개만 출력
        clean_review = review.replace("\n", " ").strip()  # 줄바꿈 제거하고 양끝 공백 제거
        short_review = clean_review[:max_length] + "..." if len(clean_review) > max_length else clean_review
        print(f"{i}. {short_review}")
        result = sentiment_analyzer(short_review)
        print(result)


Device set to use cpu


베스트셀러 5권만 출력 및 리뷰 수집

📘 2025 시대에듀 All-New KB국민은행 필기전형 봉투모의고사 9회분+무료NCS특강

📘 단 한 번의 삶
1. 이런 시기에 좋은 산문 한편 읽을 수 있게 해주셔서 고마워요
[{'label': 'neutral', 'score': 0.9998371601104736}]
2. 한동안 잠잠했던 수면장애가 다시 생겼다. 보통은 오후 열한 시쯤 잠자리에 들어서 오전 일곱 시쯤 일어나는데, 요즘은 열두 시가 넘어도 잠이 오지 않고 겨우 잠들었다 깨면 새벽 네 ...
[{'label': 'negative', 'score': 0.5167536735534668}]
3. 김영하 작가님의 에세이는 항상 저에게 새로운 패러다임을 새겨주시곤 하죠~ 한 번 읽을 때와 두 번 읽을 때와 세 번 읽을 때의 깨달음이 새록새록 다가옵니다. 단 한 번의 삶.
[{'label': 'neutral', 'score': 0.9999041557312012}]

📘 흔한남매 19
1. 좋아요~좋아요~ 좋아요~좋아요~
[{'label': 'neutral', 'score': 0.999460756778717}]
2. 좋아요~좋아요~ 좋아요~좋아요~
[{'label': 'neutral', 'score': 0.999460756778717}]
3. 딸이  좋아해요 배송도 빠르고
[{'label': 'neutral', 'score': 0.9998617172241211}]

📘 모순
1. 유명하고 잘읽힌다해서 읽어봤는데, 작가님 문체가 쉽게 잘읽혔고 매사 인생이 조용한적없는 나날이라 삶이 피곤하다생각했는데, 이 또한 경험이고 이런 여러 경험을하고 살아갈수 있는게 감...
[{'label': 'neutral', 'score': 0.9998574256896973}]
2. 이런 글을 쓰고 천천히 읽어주길 바랐다는 작가의 말이 모순이다. 안진진 어떤 선택을 하든 행복하고 불행하길 그리고 다시 행복하길 바란다.
[{'label': 'neutral', 'score': 0.99962222

##### 표 : dataFrame

In [6]:
book_data = []

for title, link in bestsellers[:10]:
    reviews = get_book_reviews(link)
    cleaned_reviews = [r.replace('\n', ' ').strip() for r in reviews[:3]]
    row = {
        "title": title,
        "reviews": " | ".join(cleaned_reviews)  # 구분 기호로 연결
    }
    book_data.append(row)

df = pd.DataFrame(book_data)
df


,title,reviews
0,결국 국민이 합니다,기다리고 기다린 책 존경하고 사랑합니다 | 이재명 대표님의 인생에는 국민이있습니다 ...
1,단 한 번의 삶,이런 시기에 좋은 산문 한편 읽을 수 있게 해주셔서 고마워요 | 한동안 잠잠했던 수...
2,대통령 윤석열 탄핵 사건 선고 결정문,헌정질서를 지켜낸 역사의 한장면을 간직하고 싶어서 구매했지요 기대됩니다.한자도 안빼...
3,행복은 언제나 당신의 편,
4,인생의 파도를 넘는 법,
5,듀얼 브레인,저는 운이 좋게도 <듀얼 브레인>을 가제본으로 먼저 읽을 수 있었습니다. 책을 정말...
6,소년이 온다,역사를 잊고 살아갈 순 없습니다. 정작 읽어야할 사람들은 안 읽는 것 같네요. | ...
7,"잠시 쉬어 가세요, 런던의 심리상담실",
8,감으로하는 브랜딩은 끝났다,차분하게 잘 읽혀서 좋아요
9,예약판매줬으면 그만이지(반양장),"제 모교 이사장님 이야기입니다. 그 분은 고교 재학시절, 그리고 현재까지 제가 늘..."
